# Uso di HDFS

Hadoop Distributed File System (HDFS) è il File System distribuito di riferimento per le applicazioni di elaborazione Big Data del cosiddetto _Ecosistema Hadoop_.

HDFS si accompagna a YARN (Yet Another Resource Negotiator) che fa da Resource Manager per la computazione distribuita sul cluster Hadoop. Un'applicazione Hadoop può essere sviluppata direttamente con le API Java ovvero attraverso uno dei molteplici framework che fanno a front-end per lo sviluppo e che mappano il proprio codice nativo in processi Java gestiti da YARN direttamente sul cluster.


## Installazione

Il prerequisito è l'installazine del Java JDK la cui cartella di installazione dovrà essere esposta esplicitamente come variabile di ambiente ```JAVA_HOME```.

I processi Hadoop hanno la necessità di connettersi in ``ssh`` senza password al server del filesystem ovvero al nodemanager. Per configurare questa opzione su ``localhost``, al fine di realizzare una configurazione a singolo nodo, eseguire i seguenti comandi nella shell:

```bash
$ ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
$ cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
$ chmod 0600 ~/.ssh/authorized_keys
````

A questo punto, si può scaricare il sorgente ```.tar.gz``` di Hadoop dalla sua [home page](https://hadoop.apache.org/releases.html) e procedere all'installazione vera e propria.

_Nota per MacOS X:_ è possibile installare Hadoop con ```brew install hadoop```


## Configurazione

La configurazione del cluster consiste nella personalizzazione di alcuni file `xml` che stabiliscono il comportamento di HDFS e YARN e che si trovano nella cartella ```/percorso/di/installazione/etc/hadoop```. In questi file andranno riportate delle proprietà di configurazione organizzate come coppie nome-valore. Di seguito si riporta la configurazione distribuita con un solo nodo all'indirizzo ``localhost``.

### ```core_site.xml```

Definisce essenzialmente l'URL del server HDFS per la connessione dei processi interessati. La web interface per l'utente si trova all'indirizzo [http://localhost:9870](http://localhost:9870) in Hadoop 3.X ovvero [http://localhost:50070](http://localhost:50070) in Hadoop 2.X.
La URL [hdfs://localhost:9099]() serve invece per la connessione al file system da parte di client e applicazioni (le porte hdfs di default sono 8020 ovvero 9000).

```xml
<configuration>
  <property>
    <name>fs.defaultFS</name>
    <value>hdfs://localhost:9099</value>
  </property>
</configuration>
```

### ```hdfs_site.xml```

Definisce il numero dei nodi e la lista di path nel filesystem locale in cui verranno memorizzate le informazioni del NameNode e dei DataNode. Con proprietà analoghe si definiscono le cartelle per i nodi di backup, checkpoint etc.

```xml
<configuration>
  <property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>
  <property>
    <name>dfs.namenode.name.dir</name>
    <value>/Users/pirrone/src/hdfs/name</value>
  </property>
  <property>
    <name>dfs.datanode.data.dir</name>
    <value>/Users/pirrone/src/hdfs/data</value>
  </property>
</configuration>
```

### ```yarn_site.xml```

Definisce il tipo di servizio del nodemanager di YARN e le variabili di ambiente di cui questo ha bisogno (sono tutte gestite direttamente dall'installazione tranne ``JAVA_HOME``). La web interface per l'utente per monitorare i processi sul cluster si trova all'indirizzo [http://localhost:8088](http://localhost:8088).

```xml
<configuration>
    <property>
        <name>yarn.nodemanager.aux-services</name>
        <value>mapreduce_shuffle</value>
    </property>
    <property>
        <name>yarn.nodemanager.env-whitelist</name>
        <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_MAPRED_HOME</value>
    </property>
</configuration>
```

### ```mapred_site.xml```

Definisce YARN quale framework di esecuzione di Map Reduce e istanzia il ``classpath`` per le applicazioni.

```xml
<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>
    <property>
        <name>mapreduce.application.classpath</name>
        <value>$HADOOP_MAPRED_HOME/share/hadoop/mapreduce/*:$HADOOP_MAPRED_HOME/share/hadoop/mapreduce/lib/*</value>
    </property>
</configuration>
```

### Avvio del cluster

Prima dell'avvio può essere opportuno configurare la variabile di ambiente ``HADOOP_HOME`` a puntare a ``/percorso/di/installazione/hadoop``. Su questa base, la prima volta che sia avvia il cluster, si deve formattare il datanode con il comando ``$HADOOP_HOME/bin/hdfs namenode -format``. Ad ogni avvio/stop del cluster e del resource manager si possono usare gli script appositi ``$HADOOP_HOME/sbin/start-all.sh`` e ``$HADOOP_HOME/sbin/stop-all.sh``.

## Uso della shell

I comandi interattivi per la shell di HDFS si possono invocare come argomenti di ``hadoop fs`` ovvero di ``hdfs dfs``. La forma è la stessa:

```bash
$ hdfs dfs -<comando> [-<opzioni_comando>] <argomenti_comando>

$ hadoop fs -<comando> [-<opzioni_comando>] <argomenti_comando>
```

In generale ``<comando>`` è uno dei tipici comandi di shell UNIX insieme con le sue opzioni. La documentazione completa si trova sul sito di [Apache Hadoop](https://hadoop.apache.org/docs/r3.3.0/hadoop-project-dist/hadoop-common/FileSystemShell.html).

## Esecuzione di applicazioni MapReduce

E' prassi creare nel filesystem HDFS delle cartelle con dei nomi ben precisi. Usualmente, si crea la cartella ``/user/<nome-utente>`` con il comando:

```bash
$ hdfs dfs -mkdir /user/<nome-utente>
```

In questa cartella ogni applicazione MapReduce gestirà una coppia di cartelle, una per l'input dei dati e una per l'output, ad esempio ``/user/<nome-utente>/<nome-app>/input`` e ``/user/<nome-utente>/<nome-app>/output`` che viene però **creata dinamicamente** dall'applicazione stessa quando viene eseguita. I dati di input saranno caricati preventivamente nella cartella di input, per esempio, con il comando:

```bash
$ hdfs dfs -put /percorso/dati/nel/fs/locale /user/<nome-utente>/<nome-app>/input/<nome-file_input>
```

Le applicazioni che usano la API Java verranno costruite sotto forma di file ``.jar`` e saranno invocate nella forma:

```bash
$ hadoop jar <nome-archivio>.jar [<nome-main-Class>] /user/<nome-utente>/<nome-app>/input /user/<nome-utente>/<nome-app>/output
```

La cartella di output viene creata dall'applicazione e popolata con i risultati.

